Мультитредінговий конвертер валют
Розробіть програму, яка використовує мультитредінг для одночасного
конвертування валют з одного формату в інший. Програма має зчитувати
валютні курси з файлу (наприклад, CSV) та конвертувати суми у різних потоках.
Реалізуйте синхронізацію між потоками для уникнення конфліктів при доступі до спільних даних.

CURRENCIES IN USE:
USD	United States Dollar 
EUR	Euro 
GBP	Pound Sterling	
UAN	Ukrainian Hryvnia	
CAD	Canadian Dollar	
CHF	Swiss Franc

In [29]:
import csv
import threading


class CurrencyConverter:
    def __init__(self, source_currency, exchange_rates):
        self.source_currency = source_currency
        self.exchange_rates = exchange_rates
        self.lock = threading.Lock()  # Додали блокування для синхронізації доступу до обмінних курсів

    def convert(self, target_currency, amount):
        if target_currency == self.source_currency:
            return amount
        with self.lock:  # Захоплюємо блокування перед доступом до обмінних курсів
            return amount * self.exchange_rates[target_currency]


def read_exchange_rates(filename):
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        exchange_rates = {}
        for row in reader:
            currency = row.pop("Currency")  # Отримуємо код валюти
            exchange_rates[currency] = {k: float(v) for k, v in row.items()}
    return exchange_rates


def main():
    filename = "exchange_rates.csv"
    exchange_rates = read_exchange_rates(filename)

    # Створюємо конвертери валют для кожної валюти.
    currency_converters = {}
    for source_currency, currency_data in exchange_rates.items():
        currency_converters[source_currency] = CurrencyConverter(source_currency, currency_data)

    # Друк доступних кодів валют для перевірки
    print("Доступні коди валют:", list(currency_converters.keys()))

    # Створюємо список потоків для конвертації валют
    conversion_threads = []

    # Конвертуємо вказані суми між різними валютами у різних потоках
    for source_currency, target_currency, amount in [
        ("USD", "EUR", 100),
        ("EUR", "PLN", 50),
        ("PLN", "UAH", 20),
        ("UAH", "GBP", 100),
        ("GBP", "USD", 75),
    ]:
        currency_converter = currency_converters.get(source_currency)
        if currency_converter is None:
            print(f"Конвертер валюти не знайдено для '{source_currency}'")
        else:
            thread = threading.Thread(target=currency_converter.convert, args=(target_currency, amount))
            conversion_threads.append(thread)
            thread.start()

    # Очікуємо завершення всіх потоків конвертації
    for thread in conversion_threads:
        thread.join()

    # Друк результатів конвертації
    for source_currency, target_currency, amount in [
        ("USD", "EUR", 100),
        ("EUR", "PLN", 50),
        ("PLN", "UAH", 20),
        ("UAH", "GBP", 100),
        ("GBP", "USD", 75),
    ]:
        currency_converter = currency_converters.get(source_currency)
        if currency_converter is not None:
            converted_amount = currency_converter.convert(target_currency, amount)
            print(f"{source_currency} to {target_currency}: {amount:.2f} -> {converted_amount:.2f}")


if __name__ == "__main__":
    main()


Доступні коди валют: ['USD', 'EUR', 'PLN', 'UAH', 'GBP']
USD to EUR: 100.00 -> 88.00
EUR to PLN: 50.00 -> 12.00
PLN to UAH: 20.00 -> 3.20
UAH to GBP: 100.00 -> 2178.00
GBP to USD: 75.00 -> 91.50
